# 10-1OpenCV套件：操作影像

In [1]:
import cv2

cv2.namedWindow("Frame", cv2.WINDOW_FULLSCREEN)
cv2.destroyWindow("Frame")

In [2]:
import cv2

try:
    img = cv2.imread("car.jpg")
    img_small = cv2.resize(img, (300, 100))
    cv2.imshow("Frame1", img)
    cv2.imshow("Frmae2", img_small)
    k = cv2.waitKey(0)#會將程式暫停
    print(k)
    cv2.destroyAllWindows()
    try:
        cv2.imwrite("small.jpg", img_small)#print可以回傳bool
        print("saved")
    except:
        print("Error: write")
except:
    print("Error: read")

97
saved


In [11]:
print(ord("q"))

113


In [5]:
#要有Frame才能有值
k = cv2.waitKey(5000)
print(k)

-1


In [7]:
#用0的話要destroy，不然會掛掉
import cv2
cv2.namedWindow("Frame", cv2.WINDOW_FULLSCREEN)
cv2.waitKey(0)
cv2.destroyWindow("Frame")

In [1]:
import cv2

capture = cv2.VideoCapture(0)
if capture.isOpened():
    while True:
        success, img = capture.read()
        if success:
            cv2.imshow("Frame", img)
        k = cv2.waitKey(100)
        if k == ord("s") or k == ord("S"):
            cv2.imwrite("shot1.jpg", img)
            print("稍後在此加入車牌辨識功能")
        
        if k == ord("q") or k == ord("Q"):
            print("exit")
            cv2.destroyAllWindows()
            capture.release()
            break
else:
    print("開啟攝影機失敗")

exit


# 10-2 Azure 電腦視覺API

## 金鑰1：4f4167ec50fe45b2a4636f903def1a40
## 金鑰2：6cb013363e234978916482925f6ef47e

In [1]:
import time
for _ in range(0, 3):
    print("Sleep...")
    time.sleep(1)
print("Wake Up!")

Sleep...
Sleep...
Sleep...
Wake Up!


In [37]:
import re
string = "Cefiro 9C-9508"
result = re.search("[\w]{2,4}", string)
print(re.findall(r"[\w]{2,4}", string))
print(result)
print(result.group())

['Cefi', 'ro', '9C', '9508']
<re.Match object; span=(0, 4), match='Cefi'>
Cefi


In [45]:
import requests
import cv2
import re
import time

base = "https://vike.cognitiveservices.azure.com/vision/v3.2/read/analyze"#com//也可以
recog_url = base#/recognizeText?mode=Printed"
#print(recog_url)
key = "4f4167ec50fe45b2a4636f903def1a40"
headers = {"Ocp-Apim-Subscription-Key" : key}
headers_stream = {"Ocp-Apim-Subscription-Key" : key, "Content-Type" : "application/octet-stream"}

img = cv2.imread("car.jpg")
# cv2.imshow("Frame", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
img_encode = cv2.imencode(".jpg", img)[1]
# print(img_encode)
img_bytes = img_encode.tobytes()
# print(img_bytes)

r1 = requests.post(recog_url, headers = headers_stream, data = img_bytes)
# r1.headers
#r1.status_code

# if r1.status_code != 202:
#     print(r1.json())
#     return "請求失敗"

result_url = r1.headers["Operation-Location"]
r2 = requests.get(result_url, headers = headers)

while r2.status_code == 200 and r2.json()["status"] != "succeeded":
    r2 = requests.get(result_url, headers = headers)
    time.sleep(0.5)
    print("status: ", r2.json()["status"])

carcard = ""
lines = r2.json()["analyzeResult"]["readResults"][0]["lines"]
#lines[0]["lines"]
# len(lines[0]["lines"])
for i in range(len(lines)):
    text = lines[i]["text"]
#     print(text)
    m = re.match(r"^[\w]{2,4}[-. ][\w]{2,4}$", text)
#     print(m)
    if m != None:
        carcard = m.group()
    
if carcard == "":
    print("找不到車牌")
else:
    print(carcard)

status:  succeeded
9C-9508


In [7]:
import requests
import cv2
import time
import re

recog_url = "https://vike.cognitiveservices.azure.com/vision/v3.2/read/analyze"
key = "4f4167ec50fe45b2a4636f903def1a40"
headers = {"Ocp-Apim-Subscription-Key" : key}
headers_stream = {"Ocp-Apim-Subscription-Key" : key, "Content-Type" : "application/octet-stream"}

def get_license(img):
    img_encode = cv2.imencode(".jpg", img)[1]
    img_bytes = img_encode.tobytes()
    r1 = requests.post(recog_url, headers = headers_stream, data = img_bytes)
    if r1.status_code != 202:
        print(r1.json())
        return "請求失敗"
    result_url = r1.headers["Operation-Location"]
    r2 = requests.get(result_url, headers = headers)
    while r2.status_code == 200 and r2.json()["status"] != "succeeded":
        r2 = requests.get(result_url, headers = headers)
        time.sleep(0.5)
        print("status: ", r2.json()["status"])
    carcard = ""
    lines = r2.json()["analyzeResult"]["readResults"][0]["lines"]
    for i in range(len(lines)):
        text = lines[i]["text"]
        m = re.match(r"^[\w]{2,4}[-. ][\w]{2,4}$", text)
        if m != None:
            carcard = m.group()
            return carcard
    if carcard == "":
        return "找不到車牌"
    
try:
    img = cv2.imread("car.jpg")
    print("status: Start")
    text = get_license(img)
    print("車牌:　", text)
    cv2.imshow("Frame", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
except:
    print("讀取圖片失敗")

status: Start
status:  running
status:  succeeded
車牌:　 9C-9508


# 10-3實戰：車牌辨識系統

In [ ]:
import license_module as m
import cv2

capture = cv2.VideoCapture(0)
if capture.isOpened():
    while True:
        success, img = capture.read()
        if success:
            cv2.imshow("Frame", img)
        k = cv2.waitKey(100)
        if k == ord("s") or k == ord("S"):
            cv2.imwrite("car4.jpg", img)
            text = m.get_license(img)
            print("車牌: ", text)
        
        if k == ord("q") or k == ord("Q"):
            print("exit")
            cv2.destroyAllWindows()
            capture.release()
            break
else:
    print("開啟攝影機失敗")

status:  running
status:  succeeded
車牌:  找不到車牌
status:  running
status:  succeeded
車牌:  找不到車牌
status:  running
status:  succeeded
車牌:  找不到車牌
status:  running
status:  succeeded
車牌:  找不到車牌
status:  succeeded
車牌:  找不到車牌
status:  running
status:  succeeded
車牌:  找不到車牌
